# Data Protection

El objetivo de este Notebook es asegurar la protección de los datos personales dentro de nuestra base de datos llamada 'Reservaciones', alineado con normas como ISO/IEC 27001 y la LFPDPPP (Ley Federal de Protección de Datos Personales en Posesión de los Particulares) 

## Sobre los datos
Nos encontramos con un dataframe con la siguiente información:

In [2]:
import pandas as pd 

rsv = pd.read_csv('/Users/manuelrodriguez/Documents/Proyectos/Clustera/Data/Reservaciones.csv')
rsv.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102154 entries, 0 to 102153
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID_Reserva              102154 non-null  int64  
 1   Fecha_hoy               102154 non-null  object 
 2   h_num_per               102154 non-null  int64  
 3   h_num_adu               102154 non-null  int64  
 4   h_num_men               102154 non-null  int64  
 5   h_num_noc               102154 non-null  int64  
 6   h_tot_hab               102154 non-null  int64  
 7   ID_Tipo_Habitacion      102154 non-null  int64  
 8   ID_Pais_Origen          102154 non-null  int64  
 9   h_edo                   102154 non-null  object 
 10  h_tfa_total             102154 non-null  float64
 11  moneda_cve              102154 non-null  int64  
 12  Hotel_cve               102154 non-null  object 
 13  Agencia_nombre          102154 non-null  object 
 14  Estado_cve          

Donde:
| Columna                | Tipo de Sensibilidad | Justificación                                                   |
|------------------------|----------------------|-----------------------------------------------------------------|
| ID_Reserva             | "Identificador"      | Puede vincularse a una persona o transacción específica.        |
| Agencia_nombre         | Sensible             | Puede revelar socios comerciales.                               |
| Empresa_nombre         | Sensible             | Identifica empresas clientes.                                   |
| Hotel_cve              | Sensible             | Información interna de operación.                               |
| Franquicia             | Sensible             | Relación con marcas específicas.                                |
| Paquete_nombre         | Sensible             | Muestra detalles de productos vendidos.                         |
| Fecha_hoy              | Temporal             | Puede usarse para inferir reservas recientes.                   |
| Estado_cve / h_edo     | "Identificador"      | Puede usarse para segmentar ubicación geográfica.               |


## Enmascaramiento de Datos 

Para proteger esta información durante el análisis, pruebas, o trabajo colaborativo, utilizaremos técnicas de enmascaramiento de datos. 

### Tokenización
La tokenización sustituye valores sensibles por identificadores únicos, preservando la estructura original, y opcionalmente permitiendo reversibilidad si se almacena un diccionario seguro.

In [4]:
import pandas as pd
import hashlib
import uuid
import os
import json

token_maps = {}

def tokenize_column(df, column):
    mapping = {}
    tokenized_values = []
    for value in df[column]:
        if value not in mapping:
            token = f"TOK_{uuid.uuid4().hex[:8]}"
            mapping[value] = token
        tokenized_values.append(mapping[value])
    token_maps[column] = mapping  # Guarda el mapeo para posible reversión
    df[column] = tokenized_values
    return df

sensitive_cols = ["Agencia_nombre", "Empresa_nombre", "Hotel_cve", "Franquicia", "Paquete_nombre"]

for col in sensitive_cols:
    rsv = tokenize_column(rsv, col)

Nuestro dataframe fue tokenizado, ahora guardaremos el mappeo en un archivo .json

In [ ]:
import json

ruta = "/Users/manuelrodriguez/Documents/Proyectos/Clustera/Outputs/token_maps.json"

with open(ruta, "w") as f:
    json.dump(token_maps, f)

Para revertir la tokenización, podemos:

In [6]:
import json
import pandas as pd

with open("/Users/manuelrodriguez/Documents/Proyectos/Clustera/Outputs/token_maps.json", "r") as f:
    token_maps = json.load(f)
for col, mapping in token_maps.items():
    reverse_map = {v: k for k, v in mapping.items()}
    rsv[col] = rsv[col].map(reverse_map)
